In [114]:
from tensorflow.keras.models import load_model
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from numpy import dot
from numpy.linalg import norm
from utils import *

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [115]:
weigths_path = '../pretrained/model.h5'
model = load_model(weigths_path)

In [116]:
def sketchify(jc):

    jc = np.uint8(jc)
    
    scale_percent = 1

    width = int(jc.shape[1]*scale_percent)
    height = int(jc.shape[0]*scale_percent)

    dim = (width,height)
    resized = cv2.resize(jc,dim,interpolation = cv2.INTER_AREA)

    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    sharpened = cv2.filter2D(resized,-1,kernel_sharpening)



    gray = cv2.cvtColor(sharpened , cv2.COLOR_BGR2GRAY)
    inv = 255-gray
    gauss = cv2.GaussianBlur(inv,ksize=(15,15),sigmaX=0,sigmaY=0)

    def dodgeV2(image,mask):
        return cv2.divide(image,255-mask,scale=256)

    pencil_jc = 255 - dodgeV2(gray,gauss)
    img = np.zeros(jc.shape)
    img[:,:,0] = pencil_jc
    img[:,:,1] = pencil_jc
    img[:,:,2] = pencil_jc
    return img

In [125]:
def get_embeedings(filepath):
    img = cv2.imread(filepath)
    img = cv2.resize(img,(256,256),interpolation = cv2.INTER_AREA)
    sketch_im = sketchify(img)/255

    output = model.predict(sketch_im[np.newaxis,:,:,:])
    class_ = output[1]
    embedding_vector = output[0][0]
    index = np.where(class_[0] > 0.05)[0]
    class_name = []
    for i in index:
        class_name.append(class_index[i])
    return class_name, embedding_vector

In [118]:
def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))

In [138]:
#sketch_file = './webapp/sample_images/test_image1.png'
#sketch_file = '../validation_image/Palladian architecture.png'
sketch_file = '../extended_database/25-sketch.jpg'
class_name, embedding_vector = get_embeedings(sketch_file)
conn, cursor = connect()
if len(class_name) == 1:
    sql_query = f"select * from image_embeddings where style in (\'{class_name[0]}\')"
else:
    sql_query = f"select * from image_embeddings where style in {str(tuple(class_name))}"
    
df = pd.read_sql(sql_query,conn)

similarity = []
for i in range(len(df)):
    embeddings = np.fromstring(df.iloc[i,3], dtype=float, sep=',')
    similarity.append(cosine_similarity(embeddings,embedding_vector))

df['similarity'] = similarity

for i in range(15):
    f = '/Users/chizhang/Downloads/archive' + df.sort_values(by = 'similarity', ascending=False).iloc[i,2]
    plt.figure()
    plt.imshow(cv2.imread(f))
    plt.axis('off')

Connected to AWS DataBase!
